In [ ]:
# === Switch to kernel bw25! ===

# Brightway libraries
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw_processing as bwp
import bw2analyzer as ba

# General libraries
import numpy as np
import pandas as pd
from fs.zipfs import ZipFS
import json                        # Library for working with json files
from pathlib import Path           # Library for working with paths in different OS     
import matplotlib.pyplot as plt    # Library for creating plots
from collections import defaultdict

In [ ]:
bd.projects.set_current('ei38-teaching-25')

In [ ]:
#if 'ei38-teaching-25' not in bd.projects:
bi.restore_project_directory(r"C:\Users\cko\sciebo1\01_CK\01_Work\1_FCN-ESE\04_Modelle\LCA_Brightway\Autumn_School\brightway2-project-ei38-teaching-25-backup.26-October-2022-03-35PM.tar.gz")

In [ ]:
eiDB = bd.Database('ei 3.8 cutoff')

In [ ]:
## CHANGE TO PROCESS WANTED! 

motor = bd.get_activity(database="ei 3.8 cutoff", name="electric motor production, vehicle (electric powertrain)")
motor

In [ ]:
el_demanded = 900 # random
electrifying_heat = 1_000_000

indices = np.array(
    [
        (electrifying_heat, electrifying_heat), # Production exchange for new motor
        (motor.id, electrifying_heat),  # Need the 
        #(el_kr.id, electrifying_heat) ## to gangnam style and positive -> because input to one of the upstream process so inherently negativ, to be 
                    
     ] + [
        (node.id,  electrifying_heat) for node in dic_powerplants_market
    ], dtype=bwp.INDICES_DTYPE
)
data = np.array([
        1,
        1,
       #el_recycling, 
    ] + [
        value * el_demanded for value in dic_powerplants_market.values()
    ]
) 
flip = np.array(
    [False, True] + [True for _ in dic_powerplants_market]
)


In [ ]:
dp = bwp.create_datapackage()

dp.add_persistent_vector(
    matrix="technosphere_matrix",
    data_array=data,
    indices_array=indices,
    flip_array=flip,
    name='electrifying_heat',
)

In [ ]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [ ]:
_, data_objs, _ = bd.prepare_lca_inputs({motor: 1}, ipcc)

In [ ]:
lca = bc.LCA({motor.id: 1}, data_objs=data_objs + [dp])
lca.lci()
lca.lcia()
lca.score

In [ ]:
lca.lcia({electrifying_heat: 1})
lca.score